# Sistemas de Recomendação - Filtragem Colaborativa

## Indice

1. [*Dataset* utilizado](#usedDataset)
2. [Extração dos dados](#dataExtraction)
3. [Sistema trivial de recomendação](#trivialSystem)
    1. [Obtendo os valores](#obtainingValues)
    2. [Recomendação por número de avaliações](#rateNumber)
    3. [Recomendação por avaliação média](#rateAverage)
4. [Sistemas de recomendação baseado em histórico](#historySystem)
    1. [Filmes similares aos já assistidos](#similarMovies)
    2. [Usuários similares ao usuário a receber recomendações](#similarUser)
        1. [Similaridade entre usuários](#similarity)
        2. [Resumindo o processo em uma função](#summary)
    3. [Sugestões baseadas em múltiplos usuários](#multipleUsers)
5. [Considerações Finais](#finalConsiderations)
    1. [Filmes de Nicho](#nicheMovies)
    2. [Filmes com poucas ou zero avaliações](#fewRatings)

Aqui exploraremos como criar, do zero, sistemas de recomendação de filmes para um usuário, como é feito em serviços de *streaming* (Netflix, Amazon Prime, etc...)

## *Dataset* utilizado <a name="usedDataset">

Para essa demonstração, foi utilizado o *dataset* ml-latest do Grouplens [encontrado neste link](grouplens.org/datasets/movielens/latest/)


## Extração dos dados <a name="dataExtraction">

Inicialmente, ao extrair os dados, é sempre importante verificar a estrutura dos mesmos. O comando `head()` nos permite ver as primeiras linhas de um *DataFrame*, e ino nosso caso isso é o suficiente para entendermos a estrutura dos dados de cada coluna

In [4]:
import pandas as pd

In [5]:
movies = pd.read_csv("ml-latest/movies.csv")
movies = movies.set_index("movieId")
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings = pd.read_csv("ml-latest/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


# Sistema trivial de recomendação <a name="trivialSystem">

É possível recomendar filmes por popularidade (quantidade de avaliações), avaliação média ou até uma heurística mista de ambos.

Embora simples, muitas vezes esses sistemas são alternativas razoáveis para apresentar para um usuário inicial, para o qual ainda não temos dados o suficiente para traçar um perfil e fazer recomendações mais embasadas.

---

## Obtendo os valores <a name="obtainingValues">

Primeiramente, vamos adicionar os valores usados para essas heurísticas do *DataFrame* de avaliações

### Número de avaliações de cada filme

In [7]:
ratings["movieId"].value_counts()

318       97999
356       97040
296       92406
593       87899
2571      84545
          ...  
182317        1
170171        1
149562        1
151521        1
127705        1
Name: movieId, Length: 53889, dtype: int64

### Avaliação média de cada filme

In [8]:
ratings.groupby("movieId").mean()["rating"]

movieId
1         3.886649
2         3.246583
3         3.173981
4         2.874540
5         3.077291
            ...   
193876    3.000000
193878    2.000000
193880    2.000000
193882    2.000000
193886    3.250000
Name: rating, Length: 53889, dtype: float64

Agora vamos adicionar esses dados ao nossos dados de filmes e verificar quais seriam os resultados de algumas heurísticas básicas.

In [9]:
movies["ratingsNum"] = ratings["movieId"].value_counts()
movies["avgRatings"] = ratings.groupby("movieId").mean()["rating"]

## Recomendação por número de avaliações <a name="rateNumber">

Embora possa parecer muito simples, os filmes mais populares, isto é, mais avaliados, também costumam ser razoavelmente bem avaliados, o que resulta em recomendações bem razoáveis, como podemos ver a seguir.

In [10]:
movies.sort_values("ratingsNum", ascending=False).head()

,title,genres,ratingsNum,avgRatings
movieId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,97999.0,4.424188
356,Forrest Gump (1994),Comedy|Drama|Romance|War,97040.0,4.056585
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,92406.0,4.173971
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,87899.0,4.151412
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,84545.0,4.149695


## Recomendação por avaliação média <a name="rateAverage">

Recomendar por avaliação média pode parecer uma boa ideia, mas de cara nos damos com um problema de muitos filmes terem uma quantidade muito baixa de avaliações para ser representativo da opinião do público geral.

É comum que filmes com avaliações média 5 (e que vão aparecer no topo da nossa lista) tenham apenas 1 ou 2 avaliações!

In [11]:
movies.sort_values("avgRatings", ascending=False).head()

,title,genres,ratingsNum,avgRatings
movieId,,,,
169338,Brad Williams: Daddy Issues (2016),Comedy,2.0,5.0
187729,Ab-normal Beauty (2004),Horror,1.0,5.0
172149,Back to You and Me (2005),Drama|Romance,1.0,5.0
160966,You're Human Like the Rest of Them (1967),(no genres listed),1.0,5.0
134387,At Ellen’s Age (2011),Comedy|Drama,1.0,5.0


Para contornar esse problema, podemos filtrar os resultados para considerar apenas filmes que já receberam um número mínimo de avaliações (no caso, escolhemos no mínimo 100 avaliações)

In [12]:
movies[movies.ratingsNum >= 100].sort_values("avgRatings", ascending=False).head()

,title,genres,ratingsNum,avgRatings
movieId,,,,
171011,Planet Earth II (2016),Documentary,853.0,4.486518
159817,Planet Earth (2006),Documentary,1384.0,4.458092
318,"Shawshank Redemption, The (1994)",Crime|Drama,97999.0,4.424188
170705,Band of Brothers (2001),Action|Drama|War,984.0,4.399898
174053,Black Mirror: White Christmas (2014),Drama|Horror|Mystery|Sci-Fi|Thriller,1074.0,4.350559


---

# Sistemas de recomendação baseado em histórico <a name="historySystem">


## Filmes similares aos já assistidos <a name="similarMovies">

Um primeiro passo é, por exemplo, dado uma lista de filmes assistidos identificar o gênero de filme mais frequentemente assistido, e recomendar filmes do mesmo gênero seguindo as heurísticas trivias já apresentadas.

In [13]:
watched = [1, 2, 3, 4, 20, 40]

In [14]:
movies.loc[watched]

,title,genres,ratingsNum,avgRatings
movieId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,68469.0,3.886649
2,Jumanji (1995),Adventure|Children|Fantasy,27143.0,3.246583
3,Grumpier Old Men (1995),Comedy|Romance,15585.0,3.173981
4,Waiting to Exhale (1995),Comedy|Drama|Romance,2989.0,2.874540
20,Money Train (1995),Action|Comedy|Crime|Drama|Thriller,4658.0,2.894483
40,"Cry, the Beloved Country (1995)",Drama,945.0,3.648148


In [15]:
genre_movies = movies[(movies.genres == "Adventure|Children|Fantasy") & (movies.ratingsNum >= 100)]
genre_movies = genre_movies.drop(watched, errors="ignore") # Removemos os filmes já assistidos das recomendações
genre_movies.sort_values("avgRatings", ascending=False).head()

,title,genres,ratingsNum,avgRatings
movieId,,,,
4896,Harry Potter and the Sorcerer's Stone (a.k.a. ...,Adventure|Children|Fantasy,27434.0,3.674892
80748,Alice in Wonderland (1933),Adventure|Children|Fantasy,117.0,3.564103
2161,"NeverEnding Story, The (1984)",Adventure|Children|Fantasy,12426.0,3.500161
50601,Bridge to Terabithia (2007),Adventure|Children|Fantasy,1987.0,3.430549
41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,13042.0,3.409216


## Usuários similares ao usuário a receber recomendações <a name="similarUser">

Também é comum recomendarmos outros filmes bem avaliados por pessoas que gostaram dos mesmos filmes que nosso usuário alvo avaliou bem. Isso pois assumimos que duas pessoas que gostaram dos mesmos filmes terão gostos parecidos.

Naturalmente, isso não é sempre verdade, mas mais à frente veremos formas de reduzir erros com esse processo!

In [16]:
def get_user_ratings(user_id: int) -> pd.DataFrame:
    """Returns all reviews from user with specified user ID"""
    user_ratings = ratings[ratings.userId == user_id]
    user_ratings = user_ratings[["movieId", "rating"]].set_index("movieId")
    
    return user_ratings

get_user_ratings(1)

,rating
movieId,
307,3.5
481,3.5
1091,1.5
1257,4.5
1449,4.5
1590,2.5
1591,1.5
2134,4.5
2478,4.0


Com `join()`, podemos criar uma tabela que, para cada filme avaliado, apresenta as notas das avaliações de ambos os usuários.

In [17]:
reviews1 = get_user_ratings(1)
reviews2 = get_user_ratings(4)

common_reviews = reviews1.join(reviews2, lsuffix="_1", rsuffix="_2").dropna()
common_reviews

,rating_1,rating_2
movieId,,
481,3.5,4.0
1091,1.5,1.0
1590,2.5,3.5
1591,1.5,3.5
2478,4.0,3.0
2840,3.0,3.5
2986,2.5,1.5
3020,4.0,3.5
3698,3.5,4.0


### Similaridade entre usuários <a name="similarity">

Tendo a tabela acima, podemos considerar a similaridade entre dois usuários como a **distância euclidiana** entre suas avaliações de cada filme.

**Caso de borda**: Também consideramos que, caso os dois usuários comparados tenham menos de 5 avaliações em comum, não há informação suficiente para determinar sua similaridade. Isso evita, por exemplo, que 2 usuários com apenas 1 avaliação em comum, mas com a mesma nota, sejam considerados como extremamente similares!

In [18]:
import numpy as np
import math

def get_user_distance(user_id1: int, user_id2:int) -> int:
    """Returns the distance between the two users """
    reviews1 = get_user_ratings(user_id1)
    reviews2 = get_user_ratings(user_id2)
    common_reviews = reviews1.join(reviews2, lsuffix="_1", rsuffix="_2").dropna()
    
    if len(common_reviews) < 5: # If there are less than N common reviews, we consider that there is not enough data to calculate an accurate distance, and so we set it to infinity
        return math.inf
    return np.linalg.norm(common_reviews["rating_1"] - common_reviews["rating_2"])

get_user_distance(1, 4)

3.0413812651491097

Tendo criada a função acima, podemos calcular a distância de um usuário com **todos os outros usuários**!

In [19]:
def all_users_distance(user_id: int, limit: int=None) -> pd.DataFrame:
    """Returns a DataFrame with the each of the distances between the specified user and all other users
    
    Can limit with how many users the distance will be calculated
    """
    all_users = ratings["userId"].unique()
    all_users = np.delete(all_users, user_id - 1)
    if limit:
        all_users=all_users[:min(limit, len(all_users))]
    
    distances = []
    for other_user in all_users:
        distances.append([user_id, other_user, get_user_distance(user_id, other_user)])
    distances_df = pd.DataFrame(distances, columns=["user1_id", "user2_id", "distance"])
    
    return distances_df

In [20]:
all_users_distance(1, limit=100).head()

,user1_id,user2_id,distance
0,1,2,inf
1,1,3,inf
2,1,4,3.041381
3,1,5,inf
4,1,6,inf


In [22]:
def get_closest_users(user_id: int, n: int=None):
    """Returns up to the n closest users to the specified user """
    distances = all_users_distance(1, limit=n).sort_values("distance")
    return distances.set_index("user2_id")

E agora vamos pegar quais são os usuários mais próximos, ou seja, os **mais similares**, ao nosso usuário alvo, descartando aqueles os quais não temos informação suficiente para determinar a proximidade.

In [19]:
closest = get_closest_users(1, 100)
closest = closest[closest.distance != math.inf]
closest

,user1_id,distance
user2_id,,
4,1,3.041381
26,1,3.640055
56,1,4.974937


Podemos, então, recomendar os filmes vistos pelo usuário mais similar seguindo uma das heurísticas triviais já apresentadas, e com isso já estaremos recomendado filmes que talvez se encaixem no perfil do nosso usuário.

In [20]:
closest_ratings = get_user_ratings(closest.iloc[0].name)
closest_ratings = closest_ratings.drop(get_user_ratings(1).index, errors="ignore") # Remove movies already watched by 1

In [21]:
recomendations = closest_ratings.sort_values("rating", ascending=False)
recomendations

,rating
movieId,
2193,5.0
27773,5.0
349,5.0
8132,5.0
8360,5.0
...,...
1732,0.5
1220,0.5
3268,0.5


### Resumindo o processo em uma função <a name="summary">

Agora que já sabemos como recomendar os filmes nos baseando no perfil do usuário mais parecido, podemos resumir o processo em uma função, para simplificar consultas futuras.

In [27]:
def get_recomendations(user_id: int, users_to_compare: int=None) -> pd.DataFrame:
    """Returns the movies seen the user that more closely resembles the specified user, sorted by the
    personal rating of this closest user
    """
    closest = get_closest_users(user_id, users_to_compare)
    closest = closest[closest.distance != math.inf] # Ignores users with no movies in common
    closest_ratings = get_user_ratings(closest.iloc[0].name)
    closest_ratings = closest_ratings.drop(get_user_ratings(1).index, errors="ignore") # Remove movies already watched by original user
    recomendations = closest_ratings.sort_values("rating", ascending=False)
    
    return recomendations.join(movies)

In [23]:
get_recomendations(1, 100)

,rating,title,genres,ratingsNum,avgRatings
movieId,,,,,
2193,5.0,Willow (1988),Action|Adventure|Fantasy,10802.0,3.456814
27773,5.0,Old Boy (2003),Mystery|Thriller,12152.0,4.070935
349,5.0,Clear and Present Danger (1994),Action|Crime|Drama|Thriller,34547.0,3.667598
8132,5.0,Gladiator (1992),Action|Drama,3853.0,3.914093
8360,5.0,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Ro...,21573.0,3.487554
...,...,...,...,...,...
1732,0.5,"Big Lebowski, The (1998)",Comedy|Crime,29805.0,3.961030
1220,0.5,"Blues Brothers, The (1980)",Action|Comedy|Musical,23154.0,3.794355
3268,0.5,Stop! Or My Mom Will Shoot (1992),Action|Comedy,2067.0,1.811079


## Sugestões baseadas em múltiplos usuários <a name="multipleUsers">

Um problema da solução anterior é que, o usuário mais próximo do nosso usuário alvo não NECESSARIAMENTE tem um gosto parecido, levando a recomendações que não agregam tanto valor.

Podemos tentar contornar esse problema levando em consideração não apenas o usuário mais similar, mas os **k usuários mais similares**, e recomendarmos filmes baseados na média de suas recomendações.

Da mesma forma que nas heurísticas introduzidas no início, vamos escolher descartar filmes que não tenham sido avaliados por vários (no caso, k/2) dos usuários mais próximos, por assumir que não temos informação suficiente para avaliar se é um filme que realmente agrada usuários com perfil parecido ou se é um filme de nicho apreciado por apenas poucos usuários.

In [28]:
def closest_users_recomendations(user_id: int, k_closest: int=None, users_to_compare: int=None) -> pd.DataFrame:
    """Returns movies that, from the k most similar users to the one specified, were rated by at least k/2 of
    them, sorted by average rating among those k users
    """
    closest = get_closest_users(user_id, users_to_compare)
    closest = closest[closest.distance != math.inf] # Ignores users with no movies in common
    if k_closest:
        closest = closest.head(k_closest)
    closest_ratings = ratings.set_index("userId").loc[closest.index] # Gets all reviews from closest users
    closest_ratings = closest_ratings.drop(get_user_ratings(1).index, errors="ignore") # Remove movies already watched by original user
    ratings_avg = closest_ratings.groupby("movieId").mean()[["rating"]] # Average rating of closest users for each movie
    ratings_amount = closest_ratings.groupby("movieId").count()[["rating"]] # Ammount of closest users that rated each movie in common
    
    recomendations = ratings_avg.join(ratings_amount, lsuffix="Avg", rsuffix="Count")
    recomendations = recomendations[recomendations.ratingCount > k_closest/2] # Removes movies rated by less than k/2 similar users
    recomendations = recomendations.sort_values("ratingAvg", ascending=False)
    
    return recomendations.join(movies).rename(columns={"ratingAvg": "similarUserRatingAvg", "ratingCount": "similarUserRatingCount"})

In [48]:
closest_users_recomendations(1, 10, 500)

,similarUserRatingAvg,similarUserRatingCount,title,genres,ratingsNum,avgRatings
movieId,,,,,,
1206,4.642857,7,"Clockwork Orange, A (1971)",Crime|Drama|Sci-Fi|Thriller,32436.0,3.981332
1625,4.583333,6,"Game, The (1997)",Drama|Mystery|Thriller,20986.0,3.855308
1208,4.500000,6,Apocalypse Now (1979),Action|Drama|War,28986.0,4.107086
2858,4.437500,8,American Beauty (1999),Drama|Romance,60820.0,4.121506
541,4.437500,8,Blade Runner (1982),Action|Sci-Fi|Thriller,39441.0,4.116097
...,...,...,...,...,...,...
2163,2.000000,6,Attack of the Killer Tomatoes! (1978),Comedy|Horror,1949.0,2.500513
1485,1.916667,6,Liar Liar (1997),Comedy,22136.0,3.228451
435,1.833333,6,Coneheads (1993),Comedy|Sci-Fi,15481.0,2.569117


## Considerações finais <a name="finalConsiderations">

Aprendemos a construir sistemas de recomendação com filtragem colaborativa, mas as técnicas aqui demonstradas também têm seus pontos cegos. Alguns exemplos de problemas são exemplificados abaixo. Com o que você aprendeu aqui, você já deve ser capaz de elaborar sistemas que solucionem esses problemas!


### Filmes de Nicho <a name="nicheMovies">

Podem existir filmes que são fortemente apreciados por uma parcela muito pequena da população, e desgostada pelo público geral. Esses tipos de filme raramente seriam recomendados pelos nossos sistemas, que estão considerando apenas as médias de votos, medida que pode facilmente descartar filmes que não sejam apreciados por grandes públicos.

### Filmes com poucas ou zero avaliações <a name="fewRatings">

Como nossas técnicas descartam filmes que não têm avaliações o suficiente, esses filmes nunca seriam recomendados, e por consequência nunca teriam a chance de receberem mais avaliações, criando um ciclo vicioso onde esses filmes nunca teriam visibilidade.